In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# the directory where your data is
PATH_to_data = '/content/drive/My Drive/STROKE_code/MMPOSE/data/patient_output/processed/'
import os
os.chdir(PATH_to_data)
import sys
sys.path.append(PATH_to_data)

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
import numpy as np
import pandas as pd

# Load the data of 3 classes
X_test3  = pd.read_csv('X_test3.csv')
X_train3 = pd.read_csv('X_train3.csv')
Y_test3  = pd.read_csv('Y_test3.csv')
Y_train3 = pd.read_csv('Y_train3.csv')

X_test3.head()


In [ ]:
Y_train3['Weakness side'].unique()

array(['L', 'none', 'R'], dtype=object)

In [ ]:
# Load the data of 2 classes (class 'none' was removed)
X_test2  = pd.read_csv('X_test2.csv')
X_train2 = pd.read_csv('X_train2.csv')
Y_test2  = pd.read_csv('Y_test2.csv')
Y_train2 = pd.read_csv('Y_train2.csv')

In [ ]:
Y_train2['Weakness side'].unique()

array(['L', 'R'], dtype=object)

In [ ]:
# comment if 34
X_test2 = X_test2.drop(['l_shoulder', 'r_shoulder'], axis=1)
X_train2 = X_train2.drop(['l_shoulder', 'r_shoulder'], axis=1)

X_test3 = X_test3.drop(['l_shoulder', 'r_shoulder'], axis=1)
X_train3 = X_train3.drop(['l_shoulder', 'r_shoulder'], axis=1)

In [ ]:
print(Y_train2.shape)
print(Y_test2.shape)

(966000, 1)
(414000, 1)


In [ ]:
print(Y_train3.shape)
print(Y_test3.shape)

(1090000, 1)
(467000, 1)


In [ ]:
#import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from os import listdir
from os.path import isfile, join, dirname, abspath

# the length of records (if shorter, we need to add some zero rows)
NUMBER_TIMESTEPS = 1000
# the number of features (from the data)
NUMBER_FEATURES = 32# 32 or 34 if you add shoulders
# the number of classes/gestures
NUMBER_OUTPUTS_2 = 2
NUMBER_OUTPUTS_3 = 3

In [ ]:
def build_model(NUMBER_OUTPUTS, NUMBER_FEATURES):
    model = models.Sequential()
    # an input layer that expects:
    # 1 or more samples, NUMBER_TIMESTEPS time steps and NUMBER_FEATURES features.

    # 1st LSTM layer
    model.add(layers.LSTM(256, return_sequences=True, input_shape=(NUMBER_TIMESTEPS, NUMBER_FEATURES)) )

    # 2nd LSTM layer
    model.add(layers.LSTM(256, input_shape=(NUMBER_TIMESTEPS, NUMBER_FEATURES)) )

    # Hidden fully connected layers of the neural network
    model.add(layers.Dense(512,activation='relu'))
    model.add(layers.Dropout(0.2))
    #model.add(layers.Dense(256,activation='relu'))
    #model.add(layers.Dropout(0.2)) #0.2 or 0.5
    #model.add(layers.Dense(512,activation='relu'))

    # Classification layer of the neural network
    model.add(layers.Dense(NUMBER_OUTPUTS, activation='softmax')) #softmax sigmoid

    opt = Adam(amsgrad=True, learning_rate=0.0000002)#, beta_1=0.9, beta_2=0.999)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', 'Precision', 'Recall']) # categorical_crossentropy #,'Precision','Recall'

    # this shows the network structure
    model.summary()

    return model

In [ ]:
# creating the model
model = build_model(NUMBER_OUTPUTS_3, NUMBER_FEATURES)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1000, 256)         295936    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 512)               131584    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 954,371
Trainable params: 954,371
Non-trainable params: 0
_________________________________________________________________


In [ ]:
Y_train2['Weakness side'].value_counts(normalize=False)

L    677000
R    289000
Name: Weakness side, dtype: int64

In [ ]:
Y_train3['Weakness side'].value_counts(normalize=False)

L       689000
R       289000
none    112000
Name: Weakness side, dtype: int64

In [ ]:
X_train2.head()

,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,Z3,X4,...,Y8,Z8,X9,Y9,Z9,X10,Y10,Z10,l_degr,r_degr
0,531.24866,665.69653,0.963227,482.26210,674.60315,0.953172,557.96857,652.33655,0.816213,446.63550,...,866.09595,0.612627,716.06146,676.82983,0.031005,388.74237,830.46936,0.839219,96.632530,19.230654
1,531.67395,665.06460,0.964282,488.33563,678.06610,0.978745,557.67694,652.06310,0.842093,449.33112,...,860.08704,0.662388,687.69190,777.74414,0.051144,392.99133,829.75024,0.847826,75.806830,25.560034
2,530.82800,668.03860,0.933524,484.64774,676.43506,0.921533,560.21533,651.24585,0.842028,446.86392,...,873.75050,0.598701,692.45874,812.87660,0.066539,388.08908,835.96670,0.851638,81.384380,19.885174
3,447.69177,535.36755,0.792352,444.60443,528.16120,0.888338,392.01334,576.18567,0.393287,384.95215,...,608.10660,1.031241,687.16660,677.63790,0.564126,497.31384,550.26250,0.827305,106.368515,132.237300
4,588.61660,622.23970,1.161708,542.77580,616.22350,1.159219,612.46390,612.01150,1.225980,530.52655,...,898.59930,1.053172,805.16600,674.86430,1.321182,516.75790,890.26447,0.905123,35.071747,56.615230


In [ ]:
# 3 classes Y
Y_train3['Weakness side'] = Y_train3['Weakness side'].replace({'L': 1, 'R': 2, 'none': 3})
Y_test3['Weakness side']   = Y_test3['Weakness side'].replace({'L': 1, 'R': 2, 'none': 3})

In [ ]:
# 2 classes Y
Y_train2['Weakness side'] = Y_train2['Weakness side'].replace({'L': 1, 'R': 0})
Y_test2['Weakness side']   = Y_test2['Weakness side'].replace({'L': 1, 'R': 0})

# one-hot encode the label column Y
Y_train_encoded = pd.get_dummies(Y_train3['Weakness side'])
Y_test_encoded  = pd.get_dummies(Y_test3 ['Weakness side'])

Y_train_encoded2 = pd.get_dummies(Y_train2['Weakness side'])
Y_test_encoded2  = pd.get_dummies(Y_test2 ['Weakness side'])

Y_test_encoded2.head()

,0,1
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [ ]:
X_train2[ ['X5','X6','X7','X8','X9','X10','Y5','Y6','Y7','Y8','Y9','Y10','r_degr','l_degr'] ].head()

,X5,X6,X7,X8,X9,X10,Y5,Y6,Y7,Y8,Y9,Y10,r_degr,l_degr
0,620.31506,415.46228,716.06146,353.11578,716.06146,388.74237,714.68300,736.94960,703.5497,866.09595,676.82983,830.46936,19.230654,96.632530
1,618.35060,414.66046,702.86030,353.98682,687.69190,392.99133,712.73676,738.73975,710.5698,860.08704,777.74414,829.75024,25.560034,75.806830
2,623.18835,413.27830,692.45874,350.30527,692.45874,388.08908,714.21890,739.40810,703.7234,873.75050,812.87660,835.96670,19.885174,81.384380
3,393.49713,384.21790,569.00555,428.66254,687.16660,497.31384,647.67130,602.14020,771.4316,608.10660,677.63790,550.26250,132.237300,106.368515
4,721.84300,527.32750,849.51337,452.64566,805.16600,516.75790,702.36830,745.32690,785.6130,898.59930,674.86430,890.26447,56.615230,35.071747


In [ ]:
Y_test2.head()

,Weakness side
0,1
1,1
2,1
3,1
4,1


In [ ]:
# count the number of NaN per column
nan_counts = X_train3.isna().sum()

print(nan_counts)

X1           0
Y1           0
Z1           0
X2           0
Y2           0
Z2           0
X3           0
Y3           0
Z3           0
X4           0
Y4           0
Z4           0
X5           0
Y5           0
Z5           0
X6           0
Y6           0
Z6           0
X7           0
Y7           0
Z7           0
X8           0
Y8           0
Z8           0
X9           0
Y9           0
Z9           0
X10          0
Y10          0
Z10          0
l_degr    1943
r_degr    1055
dtype: int64


In [ ]:
Y_train3.head()

,Weakness side
0,1
1,1
2,1
3,1
4,1


In [ ]:
# 3_X
# TRAIN
X_train3['l_degr'].fillna(90, inplace=True)
X_train3['r_degr'].fillna(90, inplace=True)
# comment bellow if no shoulder:
#X_train3['l_shoulder'].fillna(90, inplace=True)
#X_train3['r_shoulder'].fillna(90, inplace=True)

# TEST
X_test3['l_degr'].fillna(90, inplace=True)
X_test3['r_degr'].fillna(90, inplace=True)
# comment bellow if no shoulder:
#X_test3['l_shoulder'].fillna(90, inplace=True)
#X_test3['r_shoulder'].fillna(90, inplace=True)


# 2_X
# TRAIN
X_train2['l_degr'].fillna(90, inplace=True)
X_train2['r_degr'].fillna(90, inplace=True)
# comment bellow if no shoulder:
#X_train2['l_shoulder'].fillna(90, inplace=True)
#X_train2['r_shoulder'].fillna(90, inplace=True)

# TEST
X_test2['l_degr'].fillna(90, inplace=True)
X_test2['r_degr'].fillna(90, inplace=True)
# comment bellow if no shoulder:
#X_test2['l_shoulder'].fillna(90, inplace=True)
#X_test2['r_shoulder'].fillna(90, inplace=True)

In [ ]:
# count the number of NaN per column
nan_counts = X_train3.isna().sum()

print(nan_counts['l_degr'])
print(nan_counts['r_degr'])
# comment bellow if no shoulder:
#print(nan_counts['l_shoulder'])
#print(nan_counts['r_shoulder'])

0
0


In [ ]:
# comment if no shoulder

# X_train2 [ ['r_degr','l_degr', 'r_shoulder', 'l_shoulder'] ].head()

In [ ]:
has_zeros = X_train2.eq(0).any().any()

# Print the result
if has_zeros:
    print("There are values equal to 0 in the DataFrame.")
else:
    print("There are no values equal to 0 in the DataFrame.")

# Get columns with zero values
zero_columns = X_train3.columns[X_train2.eq(0).any()]

# Print the columns with zero values
print(zero_columns)

There are values equal to 0 in the DataFrame.
Index(['l_degr', 'r_degr'], dtype='object')


In [ ]:
# Get subset with values equal to 0 in any column
subset = X_train3.loc[(X_train3 == 0).any(axis=1)]

# Set the display option to show all columns
pd.set_option('display.max_columns', None)

subset.head(10)

,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,Z3,X4,Y4,Z4,X5,Y5,Z5,X6,Y6,Z6,X7,Y7,Z7,X8,Y8,Z8,X9,Y9,Z9,X10,Y10,Z10,l_degr,r_degr
349,522.31445,620.19560,0.928528,490.64233,641.31036,0.985589,532.87180,620.19560,0.617100,437.85547,662.42510,0.879995,569.82260,699.37590,0.491783,453.69153,736.32670,0.575480,559.26520,667.70380,0.127390,253.10158,688.81854,0.857729,564.54390,683.53986,0.090839,363.95392,593.80225,0.892458,0.000000,53.92584
434,502.03950,634.17444,0.930023,466.31470,655.01390,0.906964,516.92487,628.22034,0.748371,424.63574,675.85340,0.762221,566.04650,695.20435,0.288516,421.65863,732.41770,0.412193,347.23193,565.70190,0.017943,341.27780,574.63306,0.560542,566.04650,695.20435,0.020357,315.97272,504.67197,0.232628,0.000000,172.88939
533,487.90616,635.64650,0.984256,450.53085,660.56335,0.944681,506.59378,629.41724,0.889618,419.38480,679.25100,0.883864,575.11510,685.48020,0.726347,419.38480,735.31396,0.764116,668.55347,710.39710,0.376330,244.96670,685.48020,0.862202,575.11510,685.48020,0.240137,195.13293,554.66670,0.844220,0.000000,126.79984
3191,588.67760,458.39728,0.917394,559.84644,458.39728,0.871358,617.50880,487.22845,0.705952,523.80740,480.02063,0.714491,639.13220,566.51430,0.761930,523.80740,552.09863,0.665039,732.83350,660.21560,0.764859,523.80740,530.47520,0.506893,718.41797,645.80005,0.387019,545.43080,443.98170,0.689871,0.000000,165.96376
6994,508.06372,558.54175,0.951107,476.25128,572.17560,0.908638,521.69763,567.63100,0.748350,435.34960,608.53270,0.854464,576.23320,635.80050,0.700478,439.89423,685.79150,0.647381,671.67053,667.61290,0.387733,439.89423,763.05020,0.794682,748.92930,690.33606,0.226802,439.89423,653.97900,0.909656,177.954620,0.00000
7513,463.57380,578.05320,0.978895,429.82850,602.15700,0.945219,492.49835,578.05320,0.866617,415.36620,640.72310,0.868292,576.86163,619.02966,0.623910,425.00775,688.93066,0.703269,290.02650,688.93066,0.021257,275.56420,717.85520,0.870503,576.86163,619.02966,0.016756,217.71513,616.61926,0.841582,0.000000,108.79079
8018,608.69450,487.07434,0.757391,595.24960,487.07434,0.728613,649.02900,500.51917,0.654371,595.24960,527.40880,0.714739,716.25323,621.52260,0.517606,581.80475,608.07776,0.668926,743.14290,527.40880,0.456459,649.02900,473.62952,0.735438,662.47380,487.07434,0.629586,622.13934,527.40880,0.369456,100.619610,0.00000
8107,596.41504,507.53198,0.665830,573.19680,519.14110,0.639651,631.24270,542.35940,0.806002,549.97850,542.35940,0.675077,677.67920,612.01440,0.538957,538.36940,635.23267,0.816645,619.63340,472.70447,0.416462,608.02430,472.70447,0.710549,584.80590,542.35940,0.355318,573.19680,553.96850,0.242461,49.184940,0.00000
8354,602.14190,511.08698,0.817725,590.19293,523.03570,0.838067,626.03937,534.98456,0.855630,554.34660,546.93330,0.841157,697.73224,630.57500,0.640774,542.39777,630.57500,0.790195,745.52740,558.88214,0.462157,602.14190,451.34283,0.640488,649.93710,451.34283,0.381331,578.24420,523.03570,0.428248,104.676476,0.00000
8501,582.78640,535.83610,0.761109,561.42820,535.83610,0.724085,604.14440,557.19415,0.877340,529.39110,557.19415,0.722014,646.86060,642.62650,0.529632,529.39110,642.62650,0.797103,604.14440,471.76178,0.310975,561.42820,482.44086,0.847041,700.25586,514.47810,0.178701,540.07007,589.23140,0.214357,52.001198,0.00000


In [ ]:
X_train2 = X_train2.replace(0, 0.001)
X_train3 = X_train3.replace(0, 0.001)

In [ ]:
has_zeros = X_train3.eq(0).any().any()

# Print the result
if has_zeros:
    print("There are values equal to 0 in the DataFrame.")
else:
    print("There are no values equal to 0 in the DataFrame.")

# Get columns with zero values
zero_columns = X_train3.columns[X_train3.eq(0).any()]

# Print the columns with zero values
print(zero_columns)

# Get subset with values equal to 0 in any column
subset = X_train3.loc[(X_train3 == 0).any(axis=1)]

# Set the display option to show all columns
pd.set_option('display.max_columns', None)

subset.head(10)

del subset
del has_zeros

There are no values equal to 0 in the DataFrame.
Index([], dtype='object')


In [ ]:
Y_train3.head()

,Weakness side
0,1
1,1
2,1
3,1
4,1


In [ ]:
# Get unique values from column 'B'
unique_values = Y_train3['Weakness side'].unique()

# Print the unique values
print(unique_values)

del unique_values

[1 3 2]


In [ ]:
Y_train3.isnull().any()

Weakness side    False
dtype: bool

In [ ]:
'''
from sklearn.preprocessing import PowerTransformer
# Create "BoxCox" PowerTransformer scaller object
scaler = PowerTransformer(method='box-cox', standardize=True)

# Apply PowerTransformer to each column
Y_train3 = Y_train3.apply(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)))


# Apply StandardScaler to column 'B'
#Y_train3['B'] = scaler.fit_transform(Y_train3[['B']])

# Print the updated DataFrame
print(Y_train3)
'''

'\nfrom sklearn.preprocessing import PowerTransformer\n# Create "BoxCox" PowerTransformer scaller object\nscaler = PowerTransformer(method=\'box-cox\', standardize=True)\n\n# Apply PowerTransformer to each column\nY_train3 = Y_train3.apply(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)))\n\n\n# Apply StandardScaler to column \'B\'\n#Y_train3[\'B\'] = scaler.fit_transform(Y_train3[[\'B\']])\n\n# Print the updated DataFrame\nprint(Y_train3)\n'

In [ ]:
# X 3
X_train3 = np.array(X_train3)
X_test = np.array(X_test3)
# X 2
X_train2 = np.array(X_train2)
X_test2 = np.array(X_test2)

# Y 3
Y_train = np.array(Y_train_encoded)
Y_test  = np.array(Y_test_encoded)
# Y 2
Y_train2 = np.array(Y_train_encoded2)
Y_test2  = np.array(Y_test_encoded2)

print ('X 3 (train, test):')
print(X_train3.shape)
print(X_test3.shape)

print ('X 2 (train, test):')
print(X_train2.shape)
print(X_test2.shape)

print ('Y:')

print ('Y 3 (train, test):')
print(Y_train3.shape)
print(Y_test3.shape)

print ('Y 2 (train, test):')
print(Y_train2.shape)
print(Y_test2.shape)

print ('Select every 1000th row')

# Y 3
Y_train_short3 = Y_train[::NUMBER_TIMESTEPS]
Y_test_short3  = Y_test[::NUMBER_TIMESTEPS]
print ('Y 3 (train, test):')
print(Y_train_short3.shape)
print(Y_test_short3.shape)
# Y 2
Y_train_short2 = Y_train2[::NUMBER_TIMESTEPS]
Y_test_short2  = Y_test2[::NUMBER_TIMESTEPS]
print ('Y 2 short (train, test):')
print(Y_train_short2.shape)
print(Y_test_short2.shape)

X 3 (train, test):
(1090000, 32)
(467000, 32)
X 2 (train, test):
(966000, 32)
(414000, 32)
Y:
Y 3 (train, test):
(1090000, 1)
(467000, 1)
Y 2 (train, test):
(966000, 2)
(414000, 2)
Select every 1000th row
Y 3 (train, test):
(1090, 3)
(467, 3)
Y 2 short (train, test):
(966, 2)
(414, 2)


In [ ]:
# X 3
print('3 classes:')
NUMBER_BATCH_TRAIN = X_train3.shape[0] // NUMBER_TIMESTEPS
X_train3 = np.reshape(X_train3, (NUMBER_BATCH_TRAIN, NUMBER_TIMESTEPS, NUMBER_FEATURES))
print(X_test3.shape)
NUMBER_BATCH_TEST = X_test.shape[0] // NUMBER_TIMESTEPS
X_test3 = np.reshape(X_test, (NUMBER_BATCH_TEST, NUMBER_TIMESTEPS, NUMBER_FEATURES))
print(X_test3.shape)


# X 2
print('2 classes:')
NUMBER_BATCH_TRAIN2 = X_train2.shape[0] // NUMBER_TIMESTEPS
X_train2 = np.reshape(X_train2, (NUMBER_BATCH_TRAIN2, NUMBER_TIMESTEPS, NUMBER_FEATURES))
print(X_train2.shape)
NUMBER_BATCH_TEST2 = X_test2.shape[0] // NUMBER_TIMESTEPS
X_test2 = np.reshape(X_test2, (NUMBER_BATCH_TEST2, NUMBER_TIMESTEPS, NUMBER_FEATURES))
print(X_test2.shape)

del Y_test
del Y_train

3 classes:
(467000, 32)
(467, 1000, 32)
2 classes:
(966, 1000, 32)
(414, 1000, 32)


In [ ]:
print(X_train3.shape)

(1090, 1000, 32)


In [ ]:
# fit model
# history = model.fit( X_train3, Y_train_short3, validation_data=(X_test3, Y_test_short3), epochs=20, batch_size = 32, verbose = 1 ) #, callbacks=[tensorboard_callback] )#NUMBER_BATCH_TRAIN)

In [ ]:
X_train2

In [ ]:
X_train2.shape

(966, 1000, 32)

In [ ]:
X_train2[0,2,31] # batch, timestep, feature

19.885174

In [ ]:
# training the model
print(X_train3.shape)

# fit model
history3 = model.fit( X_train3, Y_train_short3, validation_data=(X_test3, Y_test_short3), epochs=100, batch_size = 128 ) #, callbacks=[tensorboard_callback] )#NUMBER_BATCH_TRAIN)

In [ ]:
'''
# training the model
print(X_train2.shape)

# fit model
history = model.fit( X_train2, Y_train_short2, validation_data=(X_test2, Y_test_short2), epochs=100, batch_size = 128 ) #, callbacks=[tensorboard_callback] )#NUMBER_BATCH_TRAIN)
'''

'\n# training the model\nprint(X_train2.shape)\n\n# fit model\nhistory = model.fit( X_train2, Y_train_short2, validation_data=(X_test2, Y_test_short2), epochs=100, batch_size = 128 ) #, callbacks=[tensorboard_callback] )#NUMBER_BATCH_TRAIN)\n'

In [ ]:
# testing the model
test_loss, test_acc, test_prec, test_rec = model.evaluate(X_test3, Y_test_short3, batch_size=128) #, test_prec, test_rec

In [ ]:
# testing the model
#test_loss, test_acc, test_prec, test_rec = model.evaluate(X_test2, Y_test_short2, batch_size=128) #, test_prec, test_rec

In [ ]:
print(history3.history)

In [ ]:
from matplotlib import pyplot
# plot loss during training
pyplot.subplot(211)


pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()


# plot accuracy during training
pyplot.subplot(212)
pyplot.title('accuracy')
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
def evaluate_model(model, x_test, y_test):
    from sklearn import metrics

    # Predict Test Data
    y_pred = model.predict(x_test)

    # Calculate accuracy, precision, recall, f1-score, and kappa score
    acc = metrics.accuracy_score(y_test, y_pred)
    prec = metrics.precision_score(y_test, y_pred)
    rec = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    kappa = metrics.cohen_kappa_score(y_test, y_pred)

    # Calculate area under curve (AUC)
    y_pred_proba = model.predict_proba(x_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)

    # Display confussion matrix
    cm = metrics.confusion_matrix(y_test, y_pred)

    return {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1,
            'kappa': kappa,
            'fpr': fpr, 'tpr': tpr, 'auc': auc,
            'cm': cm}

evaluate_model(model, X_test2, Y_test_short2)